In [ ]:

%load_ext autoreload
%matplotlib inline
%autoreload 2
%config Completer.use_jedi = True
import matplotlib as mpl
from matplotlib import pyplot as plt
mpl.rc("figure", dpi=100)
from warnings import simplefilter
# ignore all future warnings
simplefilter(action='ignore', category=FutureWarning)
import scalp
from scalp.output import draw

In [ ]:
datasets = scalp.data.loaddata_scib(scalp.test_config.scib_datapath,  maxdatasets=10,  maxcells = 1000 )
dataset = datasets[1]

# scanorama

In [ ]:
import scanpy as sc
def scanorama(dataset):
    dataset = scalp.pca.pca(dataset)
    dataset = scalp.mnn.scanorama(dataset)
    [ds.obsm.pop('umap', None)for ds in dataset]
    scalp.umapwrap.adatas_umap(dataset, label ='umap', from_obsm ='scanorama')
    return scalp.transform.stack(dataset)


sc.pl.umap(scanorama(dataset), color=['batch', 'label'])

# UMAP ONLY 

In [ ]:
import scanpy as sc
dataset = scalp.pca.pca(dataset)
[ds.obsm.pop('umap', None)for ds in dataset]
scalp.umapwrap.adatas_umap(dataset, label ='umap', from_obsm ='pca40')
stack = scalp.transform.stack(dataset)
sc.pl.umap(stack, color=['batch', 'label'])

In [ ]:
import scanpy as sc
def bbknn(dataset):
    dataset = scalp.pca.pca(dataset)
    [ds.obsm.pop('umap', None)for ds in dataset]
    dataset = scalp.mnn.bbknnwrap(dataset)
    stack = scalp.transform.stack(dataset)
    return stack

stack = bbknn(dataset)
sc.pl.umap(stack, color=['batch', 'label'])

In [ ]:
stack

In [ ]:
sc.pp.neighbors(stack,use_rep = 'pca40')
stack.obsp['connectivities']

# Combat 

In [ ]:
import scanpy as sc
[ds.obsm.pop('umap', None)for ds in dataset]
dataset = scalp.mnn.combat(dataset)
scalp.umapwrap.adatas_umap(dataset, label ='umap', from_obsm ='combat')
stack = scalp.transform.stack(dataset)
sc.pl.umap(stack, color=['batch', 'label'])

# SCALP

In [ ]:
import scanpy as sc
parm = {'intra_neigh': 8, 'intra_neighbors_mutual': False, 'inter_neigh': 4, 'add_tree': True,
              'copy_lsa_neighbors': False, 'inter_outlier_threshold': 0.95, 'inter_outlier_probabilistic_removal': False}
dataset, grap = scalp.mkgraph(dataset,**parm)
scalp.umapwrap.graph_umap(dataset,grap,label = 'umap')
stack = scalp.transform.stack(dataset)
sc.pl.umap(stack, color=['batch', 'label'])

In [ ]:

from sklearn.linear_model import SGDClassifier
def score(sad):
    X = sad.obsm['umap']
    y = sad.obsm['umap']
    
    
    